In [78]:
import json

import pandas as pd
import plotly.express as px

from ballbeam.paths import ROOT_PATH

In [79]:
sorted((ROOT_PATH / "data").glob("*.json"))

[WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/1716157543135728384.json'),
 WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/1716157921092502016.json'),
 WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/1716158556648342528.json'),
 WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/1716158878592760320.json'),
 WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/1716159110040802816.json'),
 WindowsPath('c:/Users/bjgra/GitHub/open-ball-beam/data/example.json')]

In [80]:
bases = [
    "1716157543135728384",
    "1716157921092502016",
    "1716158556648342528",
    "1716158878592760320",
    "1716159110040802816",
]

In [81]:
filename_base = bases[4]
path = ROOT_PATH / "data" / f"{filename_base}.json"
with path.open() as f:
    log_data = json.load(f)
df = pd.DataFrame(log_data)

In [82]:
# Get a list of period start indices
df["setpoint_diff"] = df["setpoint"].diff()
df["period_started"] = df["setpoint_diff"] > 0
period_idxs = df[df["period_started"]].index.to_list()

In [83]:
# Split the dataframe at each period
from itertools import pairwise
split_dfs = []
for start, stop in pairwise(period_idxs):
    split_dfs.append(df.iloc[start:stop])

In [84]:
# Write the split dataframes back out to JSON losslessly

def to_json_dict(df):
    return list(df.to_dict(orient="index").values())

for i, split_df in enumerate(split_dfs):
    out_json_data = to_json_dict(split_df)
    filename = filename_base + f"_{i:06d}" + ".json"
    out_path = ROOT_PATH / "data" / "aligned" / filename
    with out_path.open("w") as f:
        json.dump(out_json_data, f, indent=4)